In [25]:
!pip install pyod

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 137 kB 26.7 MB/s 
  Created wheel for pyod: filename=pyod-1.0.5-py3-none-any.whl size=170258 sha256=c4da32b9d794f3ad62449888ea2a89963f20a93d4de5cc313251ab56a360ccd9
  Stored in directory: /root/.cache/pip/wheels/29/98/25/bc9bfccf3907de246c61668bc09c39216ec157f8218aea4c16
Successfully built pyod


In [1]:
import tensorflow as tf

In [26]:
import math
import numpy as np
import pandas as pd
import random
import statistics
from collections import Counter
import matplotlib.pyplot as plt
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import CategoricalCrossentropy, MeanSquaredLogarithmicError
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import Accuracy, Precision, AUC, Recall
from pyod.models.copod import COPOD
from pyod.models.ecod import ECOD
from pyod.utils.data import evaluate_print

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
train1_data = pd.read_csv('/content/drive/MyDrive/X_train1_oh.csv')
train2_data = pd.read_csv('/content/drive/MyDrive/X_train1_oh.csv')
test_data  = pd.read_csv('/content/drive/MyDrive/X_test_oh.csv')

In [5]:
train1_data.shape

(140250, 385)

In [6]:
train1_data.head(5)

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,374,375,376,377,378,379,380,381,382,383
0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [7]:
train1_data=train1_data.iloc[:, 1:]

In [8]:
train1_data.shape

(140250, 384)

In [9]:
train2_data=train2_data.iloc[:, 1:]

In [10]:
train2_data.shape

(140250, 384)

In [11]:
test_data=test_data.iloc[:, 1:]

In [12]:
test_data.shape

(70126, 384)

In [13]:
from sklearn.preprocessing import MinMaxScaler

mm_scaler = MinMaxScaler()

x_train1_scaled = mm_scaler.fit_transform(train1_data)
x_train2_scaled = mm_scaler.fit_transform(train2_data)

x_test_scaled = mm_scaler.transform(test_data)

In [14]:
class AutoEncoders(Model):

  def __init__(self, output_units):

    super().__init__()
    print(output_units)
    self.encoder = Sequential(
        [
          Dense(512, activation="relu"),
          #Reducing the dimensionality of features from 218 to 32
          Dense(32, activation="relu")
        ]
    )

    self.decoder = Sequential(
        [
          Dense(256, activation="relu"),
          Dense(output_units, activation="sigmoid")
        ]
    )

  def call(self, inputs):

    encoded = self.encoder(inputs)
    decoded = self.decoder(encoded)
    return decoded
  
auto_encoder = AutoEncoders(len(x_test_scaled[0]))

auto_encoder.compile(
    loss='mean_squared_error',
    metrics=['mae', 'acc', tf.keras.metrics.Precision(), tf.keras.metrics.AUC(), tf.keras.metrics.Recall()],
    optimizer='adam'
)
callback = EarlyStopping(monitor='loss', patience=10)
history = auto_encoder.fit(
    x_train1_scaled, 
    x_train1_scaled, 
    epochs=100, 
    batch_size=2000, callbacks = [callback])
    #validation_data=(x_test_scaled, x_test_scaled)


384
Epoch 1/100
71/71 [==============================] - 8s 103ms/step - loss: 0.0542 - mae: 0.1128 - acc: 0.0000e+00 - precision: 0.6638 - auc: 0.9231 - recall: 0.7259
Epoch 2/100
71/71 [==============================] - 7s 103ms/step - loss: 0.0054 - mae: 0.0101 - acc: 0.0000e+00 - precision: 0.9943 - auc: 0.9786 - recall: 0.9510
Epoch 3/100
71/71 [==============================] - 8s 106ms/step - loss: 0.0038 - mae: 0.0056 - acc: 0.0000e+00 - precision: 0.9966 - auc: 0.9820 - recall: 0.9629
Epoch 4/100
71/71 [==============================] - 7s 103ms/step - loss: 0.0034 - mae: 0.0047 - acc: 0.0000e+00 - precision: 0.9982 - auc: 0.9832 - recall: 0.9653
Epoch 5/100
71/71 [==============================] - 8s 112ms/step - loss: 0.0032 - mae: 0.0041 - acc: 0.0000e+00 - precision: 0.9982 - auc: 0.9840 - recall: 0.9676
Epoch 6/100
71/71 [==============================] - 9s 129ms/step - loss: 0.0030 - mae: 0.0037 - acc: 0.0000e+00 - precision: 0.9987 - auc: 0.9846 - recall: 0.9697
Epoch 

In [15]:
x_train1_scaled[0]

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1.

In [16]:
auto_encoder.summary()


Model: "auto_encoders"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 32)                213536    
                                                                 
 sequential_1 (Sequential)   (None, 384)               107136    
                                                                 
Total params: 320,672
Trainable params: 320,672
Non-trainable params: 0
_________________________________________________________________


In [17]:
f2=auto_encoder.predict(x_train2_scaled)
f3=auto_encoder.predict(x_test_scaled)

2192/2192 [==============================] - 5s 2ms/step


In [18]:
encoder_layer = auto_encoder.get_layer('sequential')
f2 = pd.DataFrame(encoder_layer.predict(x_train2_scaled))
f2 = f2.add_prefix('feature_')

4383/4383 [==============================] - 8s 2ms/step


In [19]:
encoder_layer = auto_encoder.get_layer('sequential')
f3 = pd.DataFrame(encoder_layer.predict(x_test_scaled))
f3 = f3.add_prefix('feature_')

2192/2192 [==============================] - 4s 2ms/step


In [20]:
len(f2)

140250

In [21]:
f2

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31
0,14.993274,0.000000,17.224710,4.532197,17.057758,6.445744,3.369549,0.0,11.972226,13.161089,...,1.660939,0.0,13.271418,17.334667,0.729977,15.279383,0.0,14.446111,0.0,9.069098
1,14.993274,0.000000,17.224710,4.532197,17.057758,6.445744,3.369549,0.0,11.972226,13.161089,...,1.660939,0.0,13.271418,17.334667,0.729977,15.279383,0.0,14.446111,0.0,9.069098
2,11.751620,0.000000,4.367840,3.001677,13.438442,0.000000,1.433273,0.0,15.783957,11.939415,...,7.058672,0.0,17.363451,14.927039,4.474458,11.460714,0.0,22.094830,0.0,15.162474
3,14.301183,0.000000,7.873906,5.117033,0.000000,6.815405,32.167149,0.0,10.955796,6.438131,...,13.371546,0.0,8.167393,6.209394,23.738552,3.625480,0.0,3.396119,0.0,4.037393
4,14.993274,0.000000,17.224710,4.532197,17.057758,6.445744,3.369549,0.0,11.972226,13.161089,...,1.660939,0.0,13.271418,17.334667,0.729977,15.279383,0.0,14.446111,0.0,9.069098
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140245,4.751610,0.017477,0.000000,0.055501,9.245981,0.000000,2.526990,0.0,16.301779,13.392079,...,17.604113,0.0,17.694691,7.684767,18.941601,5.318397,0.0,19.266445,0.0,17.451342
140246,4.751610,0.017477,0.000000,0.055501,9.245981,0.000000,2.526990,0.0,16.301779,13.392079,...,17.604113,0.0,17.694691,7.684767,18.941601,5.318397,0.0,19.266445,0.0,17.451342
140247,6.704647,0.000000,18.665833,1.166541,13.149521,14.657135,2.183036,0.0,8.970111,13.625405,...,10.240419,0.0,11.163779,10.373122,12.464757,17.053818,0.0,8.766222,0.0,11.168348
140248,6.704647,0.000000,18.665833,1.166541,13.149521,14.657136,2.183036,0.0,8.970110,13.625406,...,10.240417,0.0,11.163780,10.373120,12.464757,17.053818,0.0,8.766221,0.0,11.168350


In [22]:
len(f3)

70126

In [23]:
f3

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31
0,6.704647,0.000000,18.665833,1.166541,13.149521,14.657135,2.183036,0.0,8.970111,13.625405,...,10.240419,0.0,11.163779,10.373122,12.464757,17.053818,0.0,8.766222,0.0,11.168348
1,4.751610,0.017477,0.000000,0.055501,9.245981,0.000000,2.526990,0.0,16.301779,13.392079,...,17.604113,0.0,17.694691,7.684767,18.941601,5.318397,0.0,19.266445,0.0,17.451342
2,11.751620,0.000000,4.367840,3.001677,13.438442,0.000000,1.433273,0.0,15.783957,11.939415,...,7.058672,0.0,17.363451,14.927039,4.474458,11.460714,0.0,22.094830,0.0,15.162474
3,14.993274,0.000000,17.224710,4.532197,17.057758,6.445744,3.369549,0.0,11.972226,13.161089,...,1.660939,0.0,13.271418,17.334667,0.729977,15.279383,0.0,14.446111,0.0,9.069098
4,6.704647,0.000000,18.665833,1.166541,13.149521,14.657135,2.183036,0.0,8.970111,13.625405,...,10.240419,0.0,11.163779,10.373122,12.464757,17.053818,0.0,8.766222,0.0,11.168348
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70121,4.751610,0.017477,0.000000,0.055501,9.245981,0.000000,2.526990,0.0,16.301779,13.392079,...,17.604113,0.0,17.694691,7.684767,18.941601,5.318397,0.0,19.266445,0.0,17.451342
70122,6.704647,0.000000,18.665833,1.166541,13.149521,14.657135,2.183036,0.0,8.970111,13.625405,...,10.240419,0.0,11.163779,10.373122,12.464757,17.053818,0.0,8.766222,0.0,11.168348
70123,17.613319,0.000000,10.355002,10.003729,1.583687,4.975470,26.919901,0.0,12.723412,10.425550,...,10.617408,0.0,9.736372,10.834968,20.398817,7.071369,0.0,7.824607,0.0,6.906029
70124,14.993271,0.000000,17.224714,4.532197,17.057756,6.445743,3.369550,0.0,11.972227,13.161089,...,1.660938,0.0,13.271420,17.334665,0.729978,15.279386,0.0,14.446116,0.0,9.069098


In [28]:
if __name__ == "__main__":
  
    # train COPOD detector
    clf_name = 'COPOD'
    clf = COPOD()

    # you could try parallel version as well.
    # clf = COPOD(n_jobs=2)
    clf.fit(f2)

    # get the prediction labels and outlier scores of the training data
    y_train_pred = clf.labels_  # binary labels (0: inliers, 1: outliers)
    y_train_scores = clf.decision_scores_  # raw outlier scores

    # get the prediction on the test data
    y_test_pred = clf.predict(f3)  # outlier labels (0 or 1)
    y_test_scores = clf.decision_function(f3)  # outlier scores

In [29]:
Counter(y_train_pred)

Counter({0: 126516, 1: 13734})

In [30]:
Counter(y_test_pred)

Counter({0: 62869, 1: 7257})

In [31]:
arr = np.array([y_test_pred])
list1 = arr.tolist()
print(list1)
print(type(list1))

Counter(y_test_pred)

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,

Counter({0: 62869, 1: 7257})

In [35]:
f3['label']=y_test_pred

In [36]:
f3.label

0        0
1        0
2        0
3        0
4        0
        ..
70121    0
70122    0
70123    1
70124    0
70125    0
Name: label, Length: 70126, dtype: int64

In [37]:
f3.shape

(70126, 33)

In [38]:
f3.head(5)

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,label
0,6.704647,0.000000,18.665833,1.166541,13.149521,14.657135,2.183036,0.0,8.970111,13.625405,...,0.0,11.163779,10.373122,12.464757,17.053818,0.0,8.766222,0.0,11.168348,0
1,4.751610,0.017477,0.000000,0.055501,9.245981,0.000000,2.526990,0.0,16.301779,13.392079,...,0.0,17.694691,7.684767,18.941601,5.318397,0.0,19.266445,0.0,17.451342,0
2,11.751620,0.000000,4.367840,3.001677,13.438442,0.000000,1.433273,0.0,15.783957,11.939415,...,0.0,17.363451,14.927039,4.474458,11.460714,0.0,22.094830,0.0,15.162474,0
3,14.993274,0.000000,17.224710,4.532197,17.057758,6.445744,3.369549,0.0,11.972226,13.161089,...,0.0,13.271418,17.334667,0.729977,15.279383,0.0,14.446111,0.0,9.069098,0
4,6.704647,0.000000,18.665833,1.166541,13.149521,14.657135,2.183036,0.0,8.970111,13.625405,...,0.0,11.163779,10.373122,12.464757,17.053818,0.0,8.766222,0.0,11.168348,0


In [41]:
#has only positive logs
p1 = f3[f3.label == 0]

In [42]:
p1.shape

(62869, 33)

In [43]:
p1.head(5)

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,label
0,6.704647,0.000000,18.665833,1.166541,13.149521,14.657135,2.183036,0.0,8.970111,13.625405,...,0.0,11.163779,10.373122,12.464757,17.053818,0.0,8.766222,0.0,11.168348,0
1,4.751610,0.017477,0.000000,0.055501,9.245981,0.000000,2.526990,0.0,16.301779,13.392079,...,0.0,17.694691,7.684767,18.941601,5.318397,0.0,19.266445,0.0,17.451342,0
2,11.751620,0.000000,4.367840,3.001677,13.438442,0.000000,1.433273,0.0,15.783957,11.939415,...,0.0,17.363451,14.927039,4.474458,11.460714,0.0,22.094830,0.0,15.162474,0
3,14.993274,0.000000,17.224710,4.532197,17.057758,6.445744,3.369549,0.0,11.972226,13.161089,...,0.0,13.271418,17.334667,0.729977,15.279383,0.0,14.446111,0.0,9.069098,0
4,6.704647,0.000000,18.665833,1.166541,13.149521,14.657135,2.183036,0.0,8.970111,13.625405,...,0.0,11.163779,10.373122,12.464757,17.053818,0.0,8.766222,0.0,11.168348,0


In [45]:
#has only negative logs
p2 = f3[f3.label == 1]

In [46]:
p2.shape

(7257, 33)

In [47]:
split_len = 0.7 * len(p1)
split_len

44008.299999999996

In [48]:
auto2 = p1[:int(split_len)]
autotest = p1[int(split_len):]

In [49]:
#This is the training set and has only positive logs
auto2.shape

(44008, 33)

In [50]:
#To reset the index
trainp = auto2.reset_index(drop=True)

In [51]:
trainp.head(5)

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,label
0,6.704647,0.000000,18.665833,1.166541,13.149521,14.657135,2.183036,0.0,8.970111,13.625405,...,0.0,11.163779,10.373122,12.464757,17.053818,0.0,8.766222,0.0,11.168348,0
1,4.751610,0.017477,0.000000,0.055501,9.245981,0.000000,2.526990,0.0,16.301779,13.392079,...,0.0,17.694691,7.684767,18.941601,5.318397,0.0,19.266445,0.0,17.451342,0
2,11.751620,0.000000,4.367840,3.001677,13.438442,0.000000,1.433273,0.0,15.783957,11.939415,...,0.0,17.363451,14.927039,4.474458,11.460714,0.0,22.094830,0.0,15.162474,0
3,14.993274,0.000000,17.224710,4.532197,17.057758,6.445744,3.369549,0.0,11.972226,13.161089,...,0.0,13.271418,17.334667,0.729977,15.279383,0.0,14.446111,0.0,9.069098,0
4,6.704647,0.000000,18.665833,1.166541,13.149521,14.657135,2.183036,0.0,8.970111,13.625405,...,0.0,11.163779,10.373122,12.464757,17.053818,0.0,8.766222,0.0,11.168348,0


In [52]:
trainp = trainp.drop(['label'], axis=1)

In [53]:
trainp.shape

(44008, 32)

In [54]:
frames = [autotest, p2]

In [55]:
autoentest = pd.concat(frames)

In [56]:
testp = autoentest.reset_index(drop=True)

In [57]:
#Has some positive logs and some negative logs and is the testing set
testp.shape

(26118, 33)

In [58]:
testp.head(5)

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,label
0,4.751610,0.017477,0.000000,0.055501,9.245981,0.000000,2.526990,0.0,16.301779,13.392079,...,0.0,17.694691,7.684767,18.941601,5.318397,0.0,19.266445,0.0,17.451342,0
1,6.704647,0.000000,18.665833,1.166541,13.149521,14.657135,2.183036,0.0,8.970111,13.625405,...,0.0,11.163779,10.373122,12.464757,17.053818,0.0,8.766222,0.0,11.168348,0
2,11.751620,0.000000,4.367840,3.001677,13.438442,0.000000,1.433273,0.0,15.783957,11.939415,...,0.0,17.363451,14.927039,4.474458,11.460714,0.0,22.094830,0.0,15.162474,0
3,11.751620,0.000000,4.367840,3.001677,13.438442,0.000000,1.433273,0.0,15.783957,11.939415,...,0.0,17.363451,14.927039,4.474458,11.460714,0.0,22.094830,0.0,15.162474,0
4,14.993274,0.000000,17.224710,4.532197,17.057758,6.445744,3.369549,0.0,11.972226,13.161089,...,0.0,13.271418,17.334667,0.729977,15.279383,0.0,14.446111,0.0,9.069098,0


In [59]:
testp = testp.drop(['label'], axis = 1)

In [60]:
testp.shape

(26118, 32)

In [61]:
len(trainp.iloc[0])

32

In [62]:
#This is the second autoencoder to train on the predicted positive and negative logs.
#This has the same architecture as the first autoencoder

In [63]:
class AutoEncoders(Model):

  def __init__(self, output_units):

    super().__init__()
    print(output_units)
    self.encoder = Sequential(
        [
          Dense(512, activation="relu"),
          #Reducing the dimensionality of features from 218 to 32
          Dense(32, activation="relu")
        ]
    )

    self.decoder = Sequential(
        [
          Dense(256, activation="relu"),
          Dense(output_units, activation="sigmoid")
        ]
    )

  def call(self, inputs):

    encoded = self.encoder(inputs)
    decoded = self.decoder(encoded)
    return decoded
  
auto_encoder2 = AutoEncoders(len(trainp.iloc[0]))

auto_encoder2.compile(
    loss='mean_squared_error',
    metrics=['mae', 'acc', tf.keras.metrics.Precision(), tf.keras.metrics.AUC(), tf.keras.metrics.Recall()],
    optimizer='adam'
)

#Must pass the training set as both input and output 
#as the encoder will learn to compress the dataset 
history = auto_encoder2.fit(
    trainp, 
    trainp, 
    epochs=60, 
    batch_size=1000,
    #validation_data=(x_test_scaled, x_test_scaled)

)


32
Epoch 1/60
45/45 [==============================] - 2s 13ms/step - loss: 92.5243 - mae: 7.3272 - acc: 0.0484 - precision_1: 0.9474 - auc_1: 0.9114 - recall_1: 0.9502
Epoch 2/60
45/45 [==============================] - 1s 14ms/step - loss: 91.8775 - mae: 7.2817 - acc: 0.0620 - precision_1: 0.9574 - auc_1: 0.9153 - recall_1: 0.9913
Epoch 3/60
45/45 [==============================] - 1s 13ms/step - loss: 91.8775 - mae: 7.2817 - acc: 0.0620 - precision_1: 0.9574 - auc_1: 0.9153 - recall_1: 0.9913
Epoch 4/60
45/45 [==============================] - 1s 13ms/step - loss: 91.8774 - mae: 7.2817 - acc: 0.0620 - precision_1: 0.9574 - auc_1: 0.9153 - recall_1: 0.9913
Epoch 5/60
45/45 [==============================] - 1s 13ms/step - loss: 91.8774 - mae: 7.2817 - acc: 0.0620 - precision_1: 0.9574 - auc_1: 0.9153 - recall_1: 0.9913
Epoch 6/60
45/45 [==============================] - 1s 13ms/step - loss: 91.8775 - mae: 7.2817 - acc: 0.0620 - precision_1: 0.9574 - auc_1: 0.9153 - recall_1: 0.9913
E

In [64]:
auto_encoder2.summary()

Model: "auto_encoders_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_2 (Sequential)   (None, 32)                33312     
                                                                 
 sequential_3 (Sequential)   (None, 32)                16672     
                                                                 
Total params: 49,984
Trainable params: 49,984
Non-trainable params: 0
_________________________________________________________________


In [66]:
encoder_layer = auto_encoder2.get_layer('sequential_2')
o1 = pd.DataFrame(encoder_layer.predict(trainp))
o1 = trainp.add_prefix('new_')

1376/1376 [==============================] - 2s 1ms/step


In [67]:
o1.shape

(44008, 32)

In [68]:
o1.head(5)

,new_feature_0,new_feature_1,new_feature_2,new_feature_3,new_feature_4,new_feature_5,new_feature_6,new_feature_7,new_feature_8,new_feature_9,...,new_feature_22,new_feature_23,new_feature_24,new_feature_25,new_feature_26,new_feature_27,new_feature_28,new_feature_29,new_feature_30,new_feature_31
0,6.704647,0.000000,18.665833,1.166541,13.149521,14.657135,2.183036,0.0,8.970111,13.625405,...,10.240419,0.0,11.163779,10.373122,12.464757,17.053818,0.0,8.766222,0.0,11.168348
1,4.751610,0.017477,0.000000,0.055501,9.245981,0.000000,2.526990,0.0,16.301779,13.392079,...,17.604113,0.0,17.694691,7.684767,18.941601,5.318397,0.0,19.266445,0.0,17.451342
2,11.751620,0.000000,4.367840,3.001677,13.438442,0.000000,1.433273,0.0,15.783957,11.939415,...,7.058672,0.0,17.363451,14.927039,4.474458,11.460714,0.0,22.094830,0.0,15.162474
3,14.993274,0.000000,17.224710,4.532197,17.057758,6.445744,3.369549,0.0,11.972226,13.161089,...,1.660939,0.0,13.271418,17.334667,0.729977,15.279383,0.0,14.446111,0.0,9.069098
4,6.704647,0.000000,18.665833,1.166541,13.149521,14.657135,2.183036,0.0,8.970111,13.625405,...,10.240419,0.0,11.163779,10.373122,12.464757,17.053818,0.0,8.766222,0.0,11.168348


In [69]:
#Compute average and standard deviation of feature values of each sample
o1['o1_mean'] = o1.mean(axis = 1) 
o1['std'] = o1.std(axis = 1)
o1_std = o1['std'] = o1.std(axis = 1)

In [70]:
o1.head(5)

,new_feature_0,new_feature_1,new_feature_2,new_feature_3,new_feature_4,new_feature_5,new_feature_6,new_feature_7,new_feature_8,new_feature_9,...,new_feature_24,new_feature_25,new_feature_26,new_feature_27,new_feature_28,new_feature_29,new_feature_30,new_feature_31,o1_mean,std
0,6.704647,0.000000,18.665833,1.166541,13.149521,14.657135,2.183036,0.0,8.970111,13.625405,...,11.163779,10.373122,12.464757,17.053818,0.0,8.766222,0.0,11.168348,8.428222,5.730798
1,4.751610,0.017477,0.000000,0.055501,9.245981,0.000000,2.526990,0.0,16.301779,13.392079,...,17.694691,7.684767,18.941601,5.318397,0.0,19.266445,0.0,17.451342,8.005532,7.108333
2,11.751620,0.000000,4.367840,3.001677,13.438442,0.000000,1.433273,0.0,15.783957,11.939415,...,17.363451,14.927039,4.474458,11.460714,0.0,22.094830,0.0,15.162474,7.399892,6.646673
3,14.993274,0.000000,17.224710,4.532197,17.057758,6.445744,3.369549,0.0,11.972226,13.161089,...,13.271418,17.334667,0.729977,15.279383,0.0,14.446111,0.0,9.069098,7.403134,6.111099
4,6.704647,0.000000,18.665833,1.166541,13.149521,14.657135,2.183036,0.0,8.970111,13.625405,...,11.163779,10.373122,12.464757,17.053818,0.0,8.766222,0.0,11.168348,8.428222,5.730798


In [71]:
#Calculating the threshold is done by multiplying std with a constant c.
#The c is chosen based on the predicted percentage of negative logs in the prediction using IF/COPOD/ECOD
threshold_value = o1_std * 0.1034

In [79]:
print(max(threshold_value))

0.7947080731391907


In [74]:
encoder_layer = auto_encoder2.get_layer('sequential_2')
o2 = pd.DataFrame(encoder_layer.predict(testp))
o2 = testp.add_prefix('new_')

817/817 [==============================] - 1s 1ms/step


In [75]:
o2.shape

(26118, 32)

In [76]:
o2.head(5)

,new_feature_0,new_feature_1,new_feature_2,new_feature_3,new_feature_4,new_feature_5,new_feature_6,new_feature_7,new_feature_8,new_feature_9,...,new_feature_22,new_feature_23,new_feature_24,new_feature_25,new_feature_26,new_feature_27,new_feature_28,new_feature_29,new_feature_30,new_feature_31
0,4.751610,0.017477,0.000000,0.055501,9.245981,0.000000,2.526990,0.0,16.301779,13.392079,...,17.604113,0.0,17.694691,7.684767,18.941601,5.318397,0.0,19.266445,0.0,17.451342
1,6.704647,0.000000,18.665833,1.166541,13.149521,14.657135,2.183036,0.0,8.970111,13.625405,...,10.240419,0.0,11.163779,10.373122,12.464757,17.053818,0.0,8.766222,0.0,11.168348
2,11.751620,0.000000,4.367840,3.001677,13.438442,0.000000,1.433273,0.0,15.783957,11.939415,...,7.058672,0.0,17.363451,14.927039,4.474458,11.460714,0.0,22.094830,0.0,15.162474
3,11.751620,0.000000,4.367840,3.001677,13.438442,0.000000,1.433273,0.0,15.783957,11.939415,...,7.058672,0.0,17.363451,14.927039,4.474458,11.460714,0.0,22.094830,0.0,15.162474
4,14.993274,0.000000,17.224710,4.532197,17.057758,6.445744,3.369549,0.0,11.972226,13.161089,...,1.660939,0.0,13.271418,17.334667,0.729977,15.279383,0.0,14.446111,0.0,9.069098


In [77]:
o2['o2_mean'] = o2.mean(axis = 1) 

In [78]:
o2.head(5)

,new_feature_0,new_feature_1,new_feature_2,new_feature_3,new_feature_4,new_feature_5,new_feature_6,new_feature_7,new_feature_8,new_feature_9,...,new_feature_23,new_feature_24,new_feature_25,new_feature_26,new_feature_27,new_feature_28,new_feature_29,new_feature_30,new_feature_31,o2_mean
0,4.751610,0.017477,0.000000,0.055501,9.245981,0.000000,2.526990,0.0,16.301779,13.392079,...,0.0,17.694691,7.684767,18.941601,5.318397,0.0,19.266445,0.0,17.451342,8.005532
1,6.704647,0.000000,18.665833,1.166541,13.149521,14.657135,2.183036,0.0,8.970111,13.625405,...,0.0,11.163779,10.373122,12.464757,17.053818,0.0,8.766222,0.0,11.168348,8.428222
2,11.751620,0.000000,4.367840,3.001677,13.438442,0.000000,1.433273,0.0,15.783957,11.939415,...,0.0,17.363451,14.927039,4.474458,11.460714,0.0,22.094830,0.0,15.162474,7.399892
3,11.751620,0.000000,4.367840,3.001677,13.438442,0.000000,1.433273,0.0,15.783957,11.939415,...,0.0,17.363451,14.927039,4.474458,11.460714,0.0,22.094830,0.0,15.162474,7.399892
4,14.993274,0.000000,17.224710,4.532197,17.057758,6.445744,3.369549,0.0,11.972226,13.161089,...,0.0,13.271418,17.334667,0.729977,15.279383,0.0,14.446111,0.0,9.069098,7.403134


In [80]:
#If the mean of the o2 dataset is less than the threshold then that row is anamoly
o2.loc[o2['o2_mean'] < 0.794708, 'label'] = 'Anamoly'  
o2.loc[o2['o2_mean'] > 0.794708, 'label'] = 'not_Anamoly'  

In [81]:
o2.head(10)

,new_feature_0,new_feature_1,new_feature_2,new_feature_3,new_feature_4,new_feature_5,new_feature_6,new_feature_7,new_feature_8,new_feature_9,...,new_feature_24,new_feature_25,new_feature_26,new_feature_27,new_feature_28,new_feature_29,new_feature_30,new_feature_31,o2_mean,label
0,4.751610,0.017477,0.000000,0.055501,9.245981,0.000000,2.526990,0.0,16.301779,13.392079,...,17.694691,7.684767,18.941601,5.318397,0.0,19.266445,0.0,17.451342,8.005532,not_Anamoly
1,6.704647,0.000000,18.665833,1.166541,13.149521,14.657135,2.183036,0.0,8.970111,13.625405,...,11.163779,10.373122,12.464757,17.053818,0.0,8.766222,0.0,11.168348,8.428222,not_Anamoly
2,11.751620,0.000000,4.367840,3.001677,13.438442,0.000000,1.433273,0.0,15.783957,11.939415,...,17.363451,14.927039,4.474458,11.460714,0.0,22.094830,0.0,15.162474,7.399892,not_Anamoly
3,11.751620,0.000000,4.367840,3.001677,13.438442,0.000000,1.433273,0.0,15.783957,11.939415,...,17.363451,14.927039,4.474458,11.460714,0.0,22.094830,0.0,15.162474,7.399892,not_Anamoly
4,14.993274,0.000000,17.224710,4.532197,17.057758,6.445744,3.369549,0.0,11.972226,13.161089,...,13.271418,17.334667,0.729977,15.279383,0.0,14.446111,0.0,9.069098,7.403134,not_Anamoly
5,6.704647,0.000000,18.665833,1.166541,13.149521,14.657135,2.183036,0.0,8.970111,13.625405,...,11.163779,10.373122,12.464757,17.053818,0.0,8.766222,0.0,11.168348,8.428222,not_Anamoly
6,6.704647,0.000000,18.665833,1.166541,13.149521,14.657135,2.183036,0.0,8.970111,13.625405,...,11.163779,10.373122,12.464757,17.053818,0.0,8.766222,0.0,11.168348,8.428222,not_Anamoly
7,4.751610,0.017477,0.000000,0.055501,9.245981,0.000000,2.526990,0.0,16.301779,13.392079,...,17.694691,7.684767,18.941601,5.318397,0.0,19.266445,0.0,17.451342,8.005532,not_Anamoly
8,6.704647,0.000000,18.665833,1.166541,13.149521,14.657135,2.183036,0.0,8.970111,13.625405,...,11.163779,10.373122,12.464757,17.053818,0.0,8.766222,0.0,11.168348,8.428222,not_Anamoly
9,6.704647,0.000000,18.665833,1.166541,13.149521,14.657135,2.183036,0.0,8.970111,13.625405,...,11.163779,10.373122,12.464757,17.053818,0.0,8.766222,0.0,11.168348,8.428222,not_Anamoly


In [83]:
result = o2[o2.label == 'not_Anamoly']

In [84]:
result.shape

(26118, 34)

In [86]:
result_anamoly = o2[o2.label == 'Anamoly']

In [87]:
result_anamoly.shape

(0, 34)